<a href="https://colab.research.google.com/github/lurodriguezzz/kaggle_/blob/main/apresent_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sklearn.preprocessing as skp
import sklearn.decomposition as skd
import sklearn.manifold as skm
import sklearn.cluster as skc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import sklearn.model_selection as skms
import sklearn.preprocessing as skpp
import sklearn.metrics as skm
import sklearn.pipeline as skpl
import sklearn.neighbors as skn
import sklearn.svm as skv
import sklearn.ensemble as ske
from sklearn.neighbors import KNeighborsClassifier
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform
from math import isnan
from scipy.stats import gmean

Construção da variável dependente, "alvo", e codificação das variáveis qualitativas binárias


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/"
df = pd.read_excel("/content/drive/My Drive/dataset_risk_3.xlsx")
df = pd.DataFrame(df)
df.head()
df['Nota do bloco quantitativo (%)'].fillna(df['Nota do bloco quantitativo (%)'].median())
df['Nota do bloco qualitativo (%)'].fillna(df['Nota do bloco qualitativo (%)'].median())
df['Nota do bloco governança (%)'].fillna(df['Nota do bloco governança (%)'].median())
media_nota = (df['Nota do bloco quantitativo (%)']+df['Nota do bloco qualitativo (%)']+df['Nota do bloco governança (%)'])/3
media_geom_nota = (df['Nota do bloco quantitativo (%)'] * df['Nota do bloco qualitativo (%)'] * df['Nota do bloco governança (%)'])**(1/3)
df.drop(['Nota do bloco quantitativo (%)', 'Nota do bloco qualitativo (%)', 'Nota do bloco governança (%)'], axis=1, inplace=True)
df['Nota do bloco'] = media_nota
df.replace('Não possui','0',inplace=True)
df.replace('Possui',1,inplace=True)
df.replace('Não atende','0',inplace=True)
df.replace('Atende',1,inplace=True)
df.replace('Sim',1,inplace=True)
df.replace('Não',0,inplace=True)
df.drop(df.columns[[0, 1, 2]], axis=1, inplace= True)
n_df = df.select_dtypes(include=['number'])
c_df = df.select_dtypes(include=['object'])
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 244066.pdf.pdf
'Aula 07 - Black & Scholes.gdrive'
'Aula Prática - Portfólio.gdrive'
 bal_MRV
 base_questoes.gsheet
 base_racional.gsheet
'CE342 - Grupo 10 - Apresentação da ata da 237ª Reunião do Copom.gslides'
'certificado_digital (3).gdoc'
'certificado_digital (3).pdf'
 Classroom
'Colab Notebooks'
 Contatos_LIGA.gdoc
'Cópia de balanco B2w.xlsx'
'Cópia de balanco.xlsx'
'Copia de VOC share price 1602-1698 (1).gsheet'
'Copia de VOC share price 1602-1698.gsheet'
'Copia de VOC share price 1602-1698 ii.gsheet'
'curriculum_lucas (1).pdf'
 curriculum_lucas.pdf
 dataset_risk_3.xlsx
 derivativos.xlsx
 DocumentoIngressante_2.pdf
'ECO-546 Aula 03 - Arquivos (1).zip'
 extensão.gdoc
 kaggle.json
 LAME4.xlsx
 lasa_trimestre.xlsx
 lasa.xlsx
 LucasAparecidodeAlmeidaRodriguez-244066.pdf
 LucasAparecidodeAlmeidaRodríguez-244066.pdf
'MEMORIA ROMA 46660001 09.09.xlsx'

,Natureza,Atividade principal,Filiais,Funcionários,2018 - Aplicações financeiras,2018 - Ativo Não Circulante Contratos Futuros,2018 - Ativo circulante,2018 - Ativo circulante Contratos Futuros,2018 - Ativo circulante sensibilizado,2018 - Ativo não circulante,...,Código de conduta,Canal de denúncia de facil acesso,Conselho consultivo ou de administração,Comitês,Auditoria externa,Conselho consultivo ou de administração e comitês específicos,Divulgação de informações,Planejamento estratégico33,Políticas formalizadas e disponíveis,Nota do bloco
0,ESTADO OU DISTRITO FEDERAL,SERVICOS DE ADMINISTRACAO PUBLICA,38.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,87.666667
1,SOCIEDADE EMPRESARIA LIMITADA,IND DE FUNDIDOS DE FERRO E ACO,0.0,168.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,0,1,0,1,1,68.000000
2,SOCIEDADE ANONIMA FECHADA,TECELAGEM EM FIBRAS NATURAIS,3.0,850.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,35.666667
3,SOCIEDADE EMPRESARIA LIMITADA,COMERCIALIZACAO DE ENERGIA ELETRICA,0.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,0,59.333333
4,SOCIEDADE ANONIMA FECHADA,USINA DE ACUCAR E ALCOOL,6.0,1231.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,73.000000


Parte qualitativa:


In [3]:
idx = c_df.columns.get_loc('Natureza')
idx_n = c_df.columns.get_loc('Atividade principal')
idx_ = idx_n+1
cat = c_df.iloc[ :,idx:idx_]
c_df = c_df.drop(c_df.iloc[:, idx:idx_].columns, axis=1)

c_df = c_df.replace(r'\D', '', regex=True)
def remove_currency(val):
    if isinstance(val, str):  # Check if the value is a string
        return val.replace("R$", "")  # Use replace for strings
    return val
c_df = c_df.applymap(remove_currency)
###
def remove_non_numeric(val):
    if isinstance(val, str):  # Check if the value is a string
        return ''.join(filter(str.isdigit, val))
    return val
c_df = c_df.applymap(remove_non_numeric)
c_df.drop(['Período analisado'], axis=1, inplace=True)


In [4]:
cat = pd.get_dummies(cat)
ncat = pd.concat( [c_df, cat], axis=1).astype(float)
ncat.head()


,Processos fiscais/criminais associados à empresa,Processos cíveis associados à empresa,Processos trabalhistas associados à empresa,Empresa inidônea,Pedidos de falência,Pedidos de recuperação judicial,Execuções judiciais,Liminar de informações,Improbidade administrativa,Registros de penalidades,...,Atividade principal_SUPERMERCADO E HIPERMERCADO,Atividade principal_Serviços,Atividade principal_Serviços de engenharia,Atividade principal_TECELAGEM EM FIBRAS NATURAIS,Atividade principal_TERMINAL DE TRANSPORTE MARITIMO,Atividade principal_TRADING,Atividade principal_TRANSPORTE FERROVIARIO E METROVIARIO,Atividade principal_TRANSPORTE RODOVIARIO DE CARGAS,"Atividade principal_TRATAMENTO DE AGUA, ESGOTO E DISTRIB DE AGUA",Atividade principal_USINA DE ACUCAR E ALCOOL
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


Quantitativo:


In [5]:

n_df_f = n_df.fillna(0)

means = n_df.mask(n_df_f == 0).mean()


#for col in n_df.columns:
 #   if n_df[col].isnull().any():
  #      n_df_f[col] = n_df_f[col].replace(0, means[col])

n_df_f = n_df_f.loc[:, (n_df_f.mean() > 0.001) | (n_df_f.mean() != 0)]
n_df_f.head()

,Filiais,Funcionários,2018 - Aplicações financeiras,2018 - Ativo Não Circulante Contratos Futuros,2018 - Ativo circulante,2018 - Ativo circulante Contratos Futuros,2018 - Ativo circulante sensibilizado,2018 - Ativo não circulante,2018 - Ativo não circulante sensibilizado,2018 - Ativo total,...,2023 - Margem líquida,2023 - Participação capital de terceiros,2023 - Patrimônio líquido31,2023 - ROE-Rentabilidade do patrimônio líquido,2023 - Receita bruta (crescimento),2023 - Índice de liquidez corrente,2023 - Índice solvência geral,Adimplente com debentures,Pontualidade em pagamentos,Nota do bloco
0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.00,0.0,1,0.70,87.666667
1,0.0,168.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.00,0.0,1,1.00,68.000000
2,3.0,850.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0711,0.0,-167760000.0,0.0,-0.0293,1.67,0.3,1,0.91,35.666667
3,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.00,0.0,1,1.00,59.333333
4,6.0,1231.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.00,0.0,1,1.00,73.000000


In [6]:
dfn = pd.concat([n_df_f, ncat], axis=1).astype(float)
dfn.drop(['Nota do bloco'], axis = 1, inplace = True)
dfn.astype(float)

,Filiais,Funcionários,2018 - Aplicações financeiras,2018 - Ativo Não Circulante Contratos Futuros,2018 - Ativo circulante,2018 - Ativo circulante Contratos Futuros,2018 - Ativo circulante sensibilizado,2018 - Ativo não circulante,2018 - Ativo não circulante sensibilizado,2018 - Ativo total,...,Atividade principal_SUPERMERCADO E HIPERMERCADO,Atividade principal_Serviços,Atividade principal_Serviços de engenharia,Atividade principal_TECELAGEM EM FIBRAS NATURAIS,Atividade principal_TERMINAL DE TRANSPORTE MARITIMO,Atividade principal_TRADING,Atividade principal_TRANSPORTE FERROVIARIO E METROVIARIO,Atividade principal_TRANSPORTE RODOVIARIO DE CARGAS,"Atividade principal_TRATAMENTO DE AGUA, ESGOTO E DISTRIB DE AGUA",Atividade principal_USINA DE ACUCAR E ALCOOL
0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,168.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,850.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6.0,1231.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413,14.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1414,523.0,91034.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1415,0.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1416,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
media_nota.astype(float)
media_nota = media_nota.fillna(media_nota.median())
media_geom_nota.astype(float)
media_geom_nota = media_geom_nota.fillna(media_geom_nota.median())

In [9]:
# Calculate correlation matrix
y=dfn
corr_matrix = y.corr()

# Define threshold and find highly correlated pairs
threshold = 0.3
columns_to_drop = set()

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            columns_to_drop.add(corr_matrix.columns[j])

# Drop the columns
y_reduced = y.drop(columns=columns_to_drop)
y_reduced.head()

,Filiais,2018 - Aplicações financeiras,2018 - Ativo Não Circulante Contratos Futuros,2018 - Contas a receber,2019 - Aplicações financeiras,2019 - Caixa e equivalentes de caixa,2019 - Debêntures (circulante),9M2021 - Ativo Não Circulante Contratos Futuros,2021 - Lucro líquido sensibilizado,1T2022 - Aplicações financeiras,...,Atividade principal_SERVICOS,Atividade principal_SERVICOS AUXILIARES DE TRANSPORTES,Atividade principal_SERVICOS DE MAO DE OBRA E ADM DE OBRAS,Atividade principal_SERVICOS DE PROCESSAMENTO DE DADOS,Atividade principal_SUPERMERCADO E HIPERMERCADO,Atividade principal_TERMINAL DE TRANSPORTE MARITIMO,Atividade principal_TRADING,Atividade principal_TRANSPORTE FERROVIARIO E METROVIARIO,Atividade principal_TRANSPORTE RODOVIARIO DE CARGAS,Atividade principal_USINA DE ACUCAR E ALCOOL
0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


Feature Selection:

In [10]:

x1 = media_nota
x2 = media_geom_nota
y = dfn
covariance_matrix = y_reduced.cov()

threshold = 0.5  # This is an example threshold, adjust based on your dataset
high_cov_pairs = {}

for i in range(covariance_matrix.shape[0]):
    for j in range(i+1, covariance_matrix.shape[1]):
        if abs(covariance_matrix.iloc[i, j]) > threshold:
            high_cov_pairs[(covariance_matrix.columns[i], covariance_matrix.columns[j])] = covariance_matrix.iloc[i, j]
drop_col = []
print("Pairs of features with high covariance:")
for pair, cov_value in high_cov_pairs.items():
    drop_col.append(pair[0])

drop_col = list(set(drop_col))
print(drop_col)

Pairs of features with high covariance:
['2019 - Dívida líquida / Patrimônio líquido', '2018 - Contas a receber', '2019 - Debêntures (circulante)', '3T2022 - Aplicações financeiras', '3T2023 - Ativo circulante', '2021 - Receita bruta (crescimento)', '2023 - Ativo circulante', '2019 - Índice de liquidez imediata sensibilizado', '2019 - Dívida líquida / Ebitda', 'Sócios controladores possuem experiência setor', '1T2022 - Ativo Não Circulante Contratos Futuros', '2021 - Dívida líquida / Ebitda', '1S2023 - Dívida total / Faturamento', '2022 - Fluxo de caixa livre / Faturamento bruto', '2020 - Giro do capital social', '2022 - Giro do capital social', '9M2021 - Ativo Não Circulante Contratos Futuros', '2021 - Índice solvência geral', 'Infrações trabalhistas', '1S2023 - Fluxo de caixa livre / Faturamento bruto', '9M2022 - Ativo circulante', '2019 - Caixa e equivalentes de caixa', 'Adimplente com debentures', 'Equipe de administração possui experiência no setor', '2021 - Lucro líquido sensibil

In [11]:
drop_col

['2019 - Dívida líquida / Patrimônio líquido',
 '2018 - Contas a receber',
 '2019 - Debêntures (circulante)',
 '3T2022 - Aplicações financeiras',
 '3T2023 - Ativo circulante',
 '2021 - Receita bruta (crescimento)',
 '2023 - Ativo circulante',
 '2019 - Índice de liquidez imediata sensibilizado',
 '2019 - Dívida líquida / Ebitda',
 'Sócios controladores possuem experiência setor',
 '1T2022 - Ativo Não Circulante Contratos Futuros',
 '2021 - Dívida líquida / Ebitda',
 '1S2023 - Dívida total / Faturamento',
 '2022 - Fluxo de caixa livre / Faturamento bruto',
 '2020 - Giro do capital social',
 '2022 - Giro do capital social',
 '9M2021 - Ativo Não Circulante Contratos Futuros',
 '2021 - Índice solvência geral',
 'Infrações trabalhistas',
 '1S2023 - Fluxo de caixa livre / Faturamento bruto',
 '9M2022 - Ativo circulante',
 '2019 - Caixa e equivalentes de caixa',
 'Adimplente com debentures',
 'Equipe de administração possui experiência no setor',
 '2021 - Lucro líquido sensibilizado',
 '2021 -

In [12]:
y_red = y_reduced.drop(columns= drop_col, axis=1)
y_red.astype(float)
y_red = pd.DataFrame(y_red)
y_red.head()

,Registros de penalidades,Trabalho escravo,Garantia financeira não aportada X calculada,Processos de monitoramento,Número de recontabilizações,Regularidade fiscal com a União,Regularidade fiscal com a Unidade Federativa,Cadeia societária,Pendências financeiras,Qualidade técnica do time,...,Atividade principal_SERVICOS,Atividade principal_SERVICOS AUXILIARES DE TRANSPORTES,Atividade principal_SERVICOS DE MAO DE OBRA E ADM DE OBRAS,Atividade principal_SERVICOS DE PROCESSAMENTO DE DADOS,Atividade principal_SUPERMERCADO E HIPERMERCADO,Atividade principal_TERMINAL DE TRANSPORTE MARITIMO,Atividade principal_TRADING,Atividade principal_TRANSPORTE FERROVIARIO E METROVIARIO,Atividade principal_TRANSPORTE RODOVIARIO DE CARGAS,Atividade principal_USINA DE ACUCAR E ALCOOL
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
x1.astype(float)
x1 = pd.DataFrame(x1)
x2.astype(float)
x2 = pd.DataFrame(x2)
x1.columns = ['x1']
x2.columns =  ['x2']
rdf = pd.concat([y_red , x1 ,x2], axis=1)
rdf.head()

,Registros de penalidades,Trabalho escravo,Garantia financeira não aportada X calculada,Processos de monitoramento,Número de recontabilizações,Regularidade fiscal com a União,Regularidade fiscal com a Unidade Federativa,Cadeia societária,Pendências financeiras,Qualidade técnica do time,...,Atividade principal_SERVICOS DE MAO DE OBRA E ADM DE OBRAS,Atividade principal_SERVICOS DE PROCESSAMENTO DE DADOS,Atividade principal_SUPERMERCADO E HIPERMERCADO,Atividade principal_TERMINAL DE TRANSPORTE MARITIMO,Atividade principal_TRADING,Atividade principal_TRANSPORTE FERROVIARIO E METROVIARIO,Atividade principal_TRANSPORTE RODOVIARIO DE CARGAS,Atividade principal_USINA DE ACUCAR E ALCOOL,x1,x2
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87.666667,87.065877
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68.000000,63.857757
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.666667,26.939967
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.333333,55.448455
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,73.000000,67.195242


In [14]:
y_red.astype(float)
x1.astype(float)
x2.astype(float)

,x2
0,87.065877
1,63.857757
2,26.939967
3,55.448455
4,67.195242
...,...
1413,84.693243
1414,88.663375
1415,61.335604
1416,58.004756


In [15]:
if y_red.shape[0] != x1.shape[0] or y_red.shape[0] != x2.shape[0]:
    raise ValueError("endog and exog matrices are different sizes")

In [16]:
# Ensure they have the same number of rows
if y_red.shape[0] != x1.shape[0] or y_red.shape[0] != x2.shape[0]:
    raise ValueError("y_red and x1 must have the same number of rows")

In [17]:
pd.set_option('display.max_rows', None)
y_red.fillna(0, inplace=True)
print(y_red.isna().sum())

Registros de penalidades                                                                                        0
Trabalho escravo                                                                                                0
Garantia financeira não aportada X calculada                                                                    0
Processos de monitoramento                                                                                      0
Número de recontabilizações                                                                                     0
Regularidade fiscal com a União                                                                                 0
Regularidade fiscal com a Unidade Federativa                                                                    0
Cadeia societária                                                                                               0
Pendências financeiras                                                                  

In [18]:
import statsmodels.formula.api as smf
import statsmodels.api as sm


y_red.astype(float)

dfn.astype(float)
#w1 = 1 / np.var(x1)
#w2 = 1 / np.var(x2)
model1 = sm.RLM(x1, y_red)
reg1 = model1.fit()

model2 = sm.RLM(x2,y_red)
reg2 = model2.fit()

Regressão:


In [19]:
print(reg1.summary())

                    Robust linear Model Regression Results                    
Dep. Variable:                     x1   No. Observations:                 1418
Model:                            RLM   Df Residuals:                     1333
Method:                          IRLS   Df Model:                           84
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Thu, 21 Mar 2024                                         
Time:                        23:17:25                                         
No. Iterations:                    25                                         
                                                                                                                   coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------

In [20]:
print(reg2.summary())

                    Robust linear Model Regression Results                    
Dep. Variable:                     x2   No. Observations:                 1418
Model:                            RLM   Df Residuals:                     1333
Method:                          IRLS   Df Model:                           84
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Thu, 21 Mar 2024                                         
Time:                        23:17:54                                         
No. Iterations:                    15                                         
                                                                                                                   coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------